In [101]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
import random


Load in the Data

In [102]:
games_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/games.csv")
games_df

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,homeFinalScore,visitorFinalScore
0,2022090800,2022,1,09/08/2022,20:20:00,LA,BUF,10,31
1,2022091100,2022,1,09/11/2022,13:00:00,ATL,NO,26,27
2,2022091101,2022,1,09/11/2022,13:00:00,CAR,CLE,24,26
3,2022091102,2022,1,09/11/2022,13:00:00,CHI,SF,19,10
4,2022091103,2022,1,09/11/2022,13:00:00,CIN,PIT,20,23
...,...,...,...,...,...,...,...,...,...
131,2022110607,2022,9,11/06/2022,13:00:00,WAS,MIN,17,20
132,2022110608,2022,9,11/06/2022,16:05:00,ARI,SEA,21,31
133,2022110609,2022,9,11/06/2022,16:25:00,TB,LA,16,13
134,2022110610,2022,9,11/06/2022,20:20:00,KC,TEN,20,17


In [115]:
plays_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/plays.csv")
plays_df = plays_df[["playDescription" , "quarter", "down", "yardsToGo", "possessionTeam","defensiveTeam","absoluteYardlineNumber", "gameClock", "preSnapHomeScore","preSnapVisitorScore","passResult", "offenseFormation", "defendersInTheBox"]]
plays_df.shape

(12486, 13)

Seperating plays by team and creating a test and train dataset for each team(later we can do analysis on which teams we can predict the best)

In [128]:
teams = plays_df.possessionTeam.unique()
#empty dictionary of team names
plays_df_by_team_test = {x : pd.DataFrame() for x in teams}
plays_df_by_team_train = {x : pd.DataFrame() for x in teams}
plays_df_test = pd.DataFrame()
plays_df_train = pd.DataFrame()
#fills dictionary with all plays according to possesion team
for team in plays_df_by_team_test.keys():
    plays_df_by_team_test[team] = plays_df[:][plays_df.possessionTeam == team]
    #seperate into train and test sets seperated by team 
    rows = len(plays_df_by_team_test[team])
    rand_idx = np.random.randint(0, rows,size = int(rows/10))
    plays_df_by_team_train[team] = plays_df_by_team_test[team].drop(plays_df_by_team_test[team].index[rand_idx])
    plays_df_by_team_test[team] = plays_df_by_team_test[team].iloc[rand_idx]
    #create a joined train and test set
    plays_df_test = pd.concat([plays_df_test, plays_df_by_team_test[team]], ignore_index = True)
    plays_df_train = pd.concat([plays_df_train, plays_df_by_team_train[team]], ignore_index = True)
    
plays_df_test

,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,absoluteYardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,offenseFormation,defendersInTheBox
0,(4:42) (Shotgun) M.Mariota pass short right to...,1,2,14,ATL,SF,66,4:42,7,0,C,SHOTGUN,6.0
1,(14:32) (Shotgun) M.Mariota pass short left to...,3,2,6,ATL,TB,32,14:32,13,0,C,SHOTGUN,6.0
2,(14:00) (Shotgun) C.Huntley right tackle to CL...,4,1,10,ATL,CLE,94,14:00,10,13,NaN,PISTOL,7.0
3,(9:30) (Shotgun) M.Mariota pass short right to...,1,3,4,ATL,SEA,79,9:30,3,0,C,SHOTGUN,7.0
4,(5:20) C.Huntley left guard to ATL 27 for 2 ya...,3,1,10,ATL,CIN,85,5:20,35,17,NaN,SINGLEBACK,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1228,(9:45) (Shotgun) J.Herbert pass short left to ...,2,3,6,LAC,JAX,81,9:45,0,13,C,SHOTGUN,5.0
1229,(5:57) (Shotgun) J.Herbert pass short left to ...,2,2,12,LAC,CLE,22,5:57,14,10,C,SHOTGUN,6.0
1230,(1:54) (Shotgun) J.Herbert pass short right to...,4,2,10,LAC,JAX,42,1:54,10,38,C,SHOTGUN,6.0
1231,(:45) (Shotgun) A.Ekeler right tackle to LV 23...,2,1,10,LAC,LV,41,0:45,10,3,NaN,SHOTGUN,5.0


In [129]:
plays_test = np.array(plays_df_test)
plays_train = np.array(plays_df_train)

In [ ]:
tackles_df = pd.read_csv("C:/Users/jesse/Desktop/CSE141L/COGS118BProject/Data/nfl-big-data-bowl-2024/tackles.csv")
tackles_df

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17423,2022091113,3642,43478,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1
